In [380]:
import pandas as pd #1.5.0
import pickle #4.0
import tensorflow as tf #2.10.0

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

In [381]:
look_up_df = pd.read_csv('look_up_table.csv')
unit_df = pd.read_csv('Unit_1.csv')
unit_df = unit_df.drop(['Unnamed: 0'], axis=1)
unit_df['net_head'] = unit_df.Magat_level - unit_df.Maris_level
models=[]
scores=[]

#### Look-up Table Evaluation

In [382]:
lk_table = unit_df[['net_head', 'U1_ActivePower']].round(2).copy()
lk_table = lk_table.merge(look_up_df, how='left')

In [383]:
X = lk_table.drop('U1_ActivePower', axis=1)
y = lk_table['U1_ActivePower']

In [384]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=123)

In [385]:
lk_table_final = X_test.copy()
lk_table_final['U1_ActivePower'] = y_test.copy()

In [386]:
model='Look-up Table'
score = mean_absolute_error(lk_table_final.U1_ActivePower, lk_table_final.unit_capability).round(3)
models.append('Look-up Table')
scores.append(score)
print(f'Model: {model}    MAE: {score}')

Model: Look-up Table    MAE: 2.073


#### Splitting of Seeded Test Set

In [387]:
unit_df = unit_df.rename(columns={'net_head': "Hydraulic_head"})

In [388]:
X = unit_df.drop('U1_ActivePower', axis=1)
y = unit_df['U1_ActivePower']

In [389]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=123)

#### XGBoost Regressor Model

In [390]:
xgb_model = pickle.load(open('saved_model/xgb_model.pkl', 'rb'))
preds = xgb_model.predict(X_test)
model = 'XGBoost'
score = mean_absolute_error(y_test, preds).round(3)
models.append(model)
scores.append(score)
print(f'Model: {model}    MAE: {score}')

Model: XGBoost    MAE: 0.324


#### LightGBM Regressor Model

In [391]:
lgbm_model = pickle.load(open('saved_model/lgbm_model.pkl', 'rb'))
preds = lgbm_model.predict(X_test)
model = 'LightGBM'
score = mean_absolute_error(y_test, preds).round(3)
models.append(model)
scores.append(score)
print(f'Model: {model}    MAE: {score}')

Model: LightGBM    MAE: 0.322


#### Histogram-based Gradient Boosting Regressor

In [392]:
hgbr_model = pickle.load(open('saved_model/hgbr_model.pkl', 'rb'))
preds = hgbr_model.predict(X_test)
model = 'HGBR'
score = mean_absolute_error(y_test, preds).round(3)
models.append(model)
scores.append(score)
print(f'Model: {model}    MAE: {score}')

Model: HGBR    MAE: 0.327


#### Gradient Boosting Regressor

In [393]:
gbr_model = pickle.load(open('saved_model/gbr_model.pkl', 'rb'))
preds = gbr_model.predict(X_test)
model = 'GBR'
score = mean_absolute_error(y_test, preds).round(3)
models.append(model)
scores.append(score)
print(f'Model: {model}    MAE: {score}')

Model: GBR    MAE: 0.319


#### Ensemble Model

In [394]:
ensemble_model = pickle.load(open('saved_model/ensemble_model.pkl', 'rb'))
preds = ensemble_model.predict(X_test)
model = 'Ensemble Model'
score = mean_absolute_error(y_test, preds).round(3)
models.append(model)
scores.append(score)
print(f'Model: {model}    MAE: {score}')

Model: Ensemble Model    MAE: 0.318


#### Deep Neural Network

In [395]:
dnn_X_test = X_test.drop('Hydraulic_head', axis=1)

In [396]:
dnn_model = tf.keras.models.load_model('saved_model/dnn_unit1')
preds = dnn_model.predict(dnn_X_test)
model = 'DNN'
score = mean_absolute_error(y_test, preds).round(3)
models.append(model)
scores.append(score)
print(f'Model: {model}    MAE: {score}')

341/341 [==============================] - 0s 1ms/step
Model: DNN    MAE: 0.327


In [397]:
model_eval = pd.DataFrame({'Model': models, 'MAE': scores})
model_eval = model_eval.sort_values(by=['MAE'])
percent_reduc = (abs(model_eval.MAE - 2.073)/2.073) * 100
model_eval['% Error Reduction'] = percent_reduc.round(2)
model_eval

,Model,MAE,% Error Reduction
5,Ensemble Model,0.318,84.66
4,GBR,0.319,84.61
2,LightGBM,0.322,84.47
1,XGBoost,0.324,84.37
3,HGBR,0.327,84.23
6,DNN,0.327,84.23
0,Look-up Table,2.073,0.00
